# IMPORTANDO LIBRERIAS

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../Data/datos_seattle.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30657 entries, 0 to 30656
Data columns (total 46 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   osebuildingid                   30657 non-null  int64  
 1   datayear                        30657 non-null  int64  
 2   buildingname                    30657 non-null  object 
 3   buildingtype                    30657 non-null  object 
 4   taxparcelidentificationnumber   30657 non-null  object 
 5   address                         30657 non-null  object 
 6   city                            30657 non-null  object 
 7   state                           30657 non-null  object 
 8   zipcode                         30657 non-null  int64  
 9   latitude                        30657 non-null  float64
 10  longitude                       30657 non-null  float64
 11  neighborhood                    30657 non-null  object 
 12  councildistrictcode             

In [4]:
df.isnull().sum().sort_values(ascending=False)

steamuse_kbtu                     29600
thirdlargestpropertyusetypegfa    24129
thirdlargestpropertyusetype       23968
secondlargestpropertyusetype      13143
secondlargestpropertyuse          13143
naturalgas_kbtu                   11489
naturalgas_therms                 11489
energystarscore                    7958
siteeuiwn_kbtu_sf                  1207
sourceeuiwn_kbtu_sf                1207
siteenergyusewn_kbtu               1202
siteeui_kbtu_sf                     897
sourceeui_kbtu_sf                   897
siteenergyuse_kbtu                  892
electricity_kwh                     581
electricity_kbtu                    580
ghgemissionsintensity               500
totalghgemissions                   498
complianceissue                     326
numberofbuildings                   294
numberoffloors                       83
largestpropertyusetype               47
largestpropertyusetypegfa            42
epapropertytype                      39
address                               0
